In [60]:
import numpy as np
import pandas as pd
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.util import bigrams, trigrams, ngrams
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
df = pd.read_csv("../englishComments.csv")
display(df)

,Username,Comment,Likes,Video ID
0,Pardeep Sandhu,Mam actually ek kisan ki death ho gyi hai usko...,0,0lrE3zoaCqI
1,Sahab Singh,Ko bi,0,0lrE3zoaCqI
2,Gurlal singh,Very bad BJP cm jjp,0,0lrE3zoaCqI
3,pc bansal,Kisan kalank Tikait ko bhejo jail.,0,0lrE3zoaCqI
4,harman bajwa,Kisan zindabad,0,0lrE3zoaCqI
...,...,...,...,...
40785,Rahul Kushwaha,Ese dekh MEWAR ki yaad aa rhi,0,jTrlAzXZJ5M
40786,Shreya Raj,May the moral of Panjshir warriors stay high ....,3,jTrlAzXZJ5M
40787,Monindra Mahato,INDIA ko vi Saran nahi dena hay,0,jTrlAzXZJ5M
40788,Akash Isadkar,Ha aur taliban ke 300 ladko Maar dala ye bhi b...,0,jTrlAzXZJ5M


In [62]:
# text filtration
def filteredTokens(line):

    # remove stop words
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(line)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    punctuations = re.compile(r'[-,*%#&"\'`:+.?!;()|0-9]')    # remove punctuations
    post_punctuations = []
    for words in filtered_sentence:
        word = punctuations.sub("",words)
        if len(word)>0:
            post_punctuations.append(word)
    
    return post_punctuations
    

In [63]:
# Python program for KMP Algorithm (works for both string and list) gfg
def KMPSearch(pat, txt):
	M = len(pat)
	N = len(txt)
	output = []

	# create lps[] that will hold the longest prefix suffix
	# values for pattern
	lps = [0]*M
	j = 0 # index for pat[]

	# Preprocess the pattern (calculate lps[] array)
	computeLPSArray(pat, M, lps)

	i = 0 # index for txt[]
	while i < N:
		if pat[j] == txt[i]:
			i += 1
			j += 1

		if j == M:
			output.append(i-j)
			j = lps[j-1]

		# mismatch after j matches
		elif i < N and pat[j] != txt[i]:
			# Do not match lps[0..lps[j-1]] characters,
			# they will match anyway
			if j != 0:
				j = lps[j-1]
			else:
				i += 1
	
	return output

def computeLPSArray(pat, M, lps):
	len = 0 # length of the previous longest prefix suffix

	lps[0] # lps[0] is always 0
	i = 1

	# the loop calculates lps[i] for i = 1 to M-1
	while i < M:
		if pat[i] == pat[len]:
			len += 1
			lps[i] = len
			i += 1
		else:
			# This is tricky. Consider the example.
			# AAACAAAA and i = 7. The idea is similar
			# to search step.
			if len != 0:
				len = lps[len-1]

				# Also, note that we do not increment i here
			else:
				lps[i] = 0
				i += 1

In [64]:
# finds all occurances of next word after target (string) in source (list)
def nextWordOfPhrase(target, source):
    
    target = word_tokenize(target)
    m = len(target)
    n = len(source)

    lst = KMPSearch(target,source)      # search all occurances of target
 
    nextWords = []
    # all next words
    for val in lst:
        if(m+val) < n:
            nextWords.append(source[m+val])
    
    return nextWords

## Farmers are and Farmers are not

In [58]:
cloud1 = []
cloud2 = []

for i in range(len(df)):
    line = df.Comment[i]
    line = str(line)
    tokens = filteredTokens(line.lower())       # remove unnecessary words & punc.
    p1 = nextWordOfPhrase('kisan',tokens)
    p2 = nextWordOfPhrase('kisan nahi',tokens)
    cloud1.extend(p1)
    cloud2.extend(p2)

In [59]:
# save in csv
set1, count1 = np.unique(cloud1, return_counts=True)
set2, count2 = np.unique(cloud2, return_counts=True)

pd.DataFrame({'set1': set1, 'frequency': count1}).to_csv('perception_set1.csv', index=False)
pd.DataFrame({'set2': set2, 'frequency': count2}).to_csv('perception_set2.csv', index=False)
